In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from random import shuffle
import matplotlib.pyplot as plt
import cv2
print(os.listdir("../input"))

In [ ]:
#training data set directory
traindir = "../input/train"
#testing data set directory
testdir = '..input/test'
img_size = 50
learning_rate = 0.001

In [ ]:
#[1,0] is for cat and [0,1] is for dog.
def label_image(image):
    label = image.split('.')[0]
    if label == 'cat':
        return [1,0]
    elif label == 'dog':
        return [0,1]

In [ ]:
def create_training_data():
    training_data = []
    for img in os.listdir(traindir):
        label = label_image(img) 
        path = os.path.join(traindir,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE) #convert the image into gray scale
        img = cv2.resize(img,(img_size,img_size))
        training_data.append([img,np.array([label])])
    shuffle(training_data)    
    return training_data

In [ ]:
def creat_test_data():
    test_data=[]
    for img in os.listdir(traindir):
        path=os.path.join(testdir,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(img_size,img_size))
        test_data.append([img,img_num])
    shuffle(test_data)    
    return test_data

In [ ]:
train_data = create_training_data()

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected,flatten
from tflearn.layers.estimator import regression


convnet = input_data(shape=[None,img_size,img_size,1],name='input')

#model architecture
convnet = conv_2d(convnet,128,3,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,3,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,3,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = flatten(convnet)
convnet = fully_connected(convnet,1024,activation='relu')

convnet = fully_connected(convnet,256,activation='relu')

convnet = fully_connected(convnet,128,activation='relu')

convnet = fully_connected(convnet,2,activation='relu')

convnet = regression(convnet,optimizer='adam',learning_rate=learning_rate,loss='categorical_crossentropy', name='targets')


In [ ]:
model = tflearn.DNN(convnet,tensorboard_dir='log')

In [ ]:
train = train_data[0:-500]
val = train_data[-500:0]

In [ ]:
#training set
X = np.array([i[0] for i in train]).reshape([-1,img_size,img_size,1])
y = np.array([i[1] for i in train])
val_x = np.array([i[0] for i in val]).reshape([-1,img_size,img_size,1])
val_y = np.array([i[1] for i in val])

In [ ]:
model.fit({'input':X}, {'targets': y}, n_epoch=3, validation_set=({'input': val_x}, {'targets': val_y}), snapshot_step=500, show_metric=True)

In [ ]:
fig=plt.figure()

for num,data in enumerate(test_data[:15]):    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(3,5,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: 
        str_label='Dog'
    else: 
        str_label='Cat'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()